In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from mindspore import context, Tensor, nn, ops
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.common import dtype as mstype
from mindspore.train import Model, Callback
from mindspore.train.callback import LossMonitor, TimeMonitor, ModelCheckpoint, CheckpointConfig
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits
from mindspore.nn import Accuracy
from sklearn.preprocessing import StandardScaler
from download import download


import os
import pickle
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt 

pgf = False
if (pgf):
    matplotlib.use("pgf")
    matplotlib.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,
    })

In [2]:
cifar10_url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/datasets/cifar-10-binary.tar.gz"
download(cifar10_url, "./dataset", kind="tar.gz", replace=False)

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.


'./dataset'

In [3]:
#HYPERPARAMETERS
hidden_units_large = [512, 512]
hidden_units_med = [256, 256]
hidden_units_low = [64, 64]
hidden_units_verylow = [32, 32]

epochs = 20
nb_classes = 10
input_shape = (32, 32, 3)
dropout = [0.25, 0.25, 0.25]
batch_size = 64

In [4]:
def load_cifar10(batch_size=64):
    # Download CIFAR-10 dataset using MindSpore dataset module
    dataset_train = ds.Cifar10Dataset("./dataset", usage="train", shuffle=True)
    dataset_test = ds.Cifar10Dataset("./dataset", usage="test", shuffle=False)

    trans = [
        CV.Rescale(1.0 / 255.0, 0.0),
        CV.HWC2CHW()
    ]
    type_cast_op = C.TypeCast(mstype.int32)

    dataset_train = dataset_train.map(operations=trans, input_columns="image")
    dataset_train = dataset_train.map(operations=type_cast_op, input_columns="label")
    dataset_train = dataset_train.batch(batch_size)

    dataset_test = dataset_test.map(operations=trans, input_columns="image")
    dataset_test = dataset_test.map(operations=type_cast_op, input_columns="label")
    dataset_test = dataset_test.batch(batch_size)

    return dataset_train, dataset_test

train_ds, test_ds = load_cifar10(batch_size=batch_size)

[WARNING] ME(36097:128790985704960,MainProcess):2025-07-30-17:48:49.700.000 [mindspore/dataset/core/validator_helpers.py:744] 'Rescale' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Rescale' from mindspore.dataset.vision instead.
[WARNING] ME(36097:128790985704960,MainProcess):2025-07-30-17:48:49.701.000 [mindspore/dataset/core/validator_helpers.py:744] 'HWC2CHW' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'HWC2CHW' from mindspore.dataset.vision instead.
[WARNING] ME(36097:128790985704960,MainProcess):2025-07-30-17:48:49.701.000 [mindspore/dataset/core/validator_helpers.py:744] 'TypeCast' from mindspore.dataset.transforms.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'TypeCast' from mindspore.dataset.transforms instead.


In [5]:
class MLP(nn.Cell):
    def __init__(self, input_size=input_shape[0] * input_shape[1] * input_shape[2], hidden_sizes=hidden_units_large, num_classes=nb_classes):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Dense(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Dense(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.out = nn.Dense(hidden_sizes[1], num_classes)

    def construct(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.out(x)

In [6]:
net = MLP()
loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = nn.Adam(net.trainable_params(), learning_rate=0.001)
model = Model(net, loss_fn=loss_fn, optimizer=optimizer, metrics={"Accuracy": Accuracy()})
print(net)

# Callbacks
loss_cb = LossMonitor()
time_cb = TimeMonitor()

# Training
model.train(epoch=epochs, train_dataset=train_ds, callbacks=[loss_cb, time_cb], dataset_sink_mode=False)


MLP(
  (flatten): Flatten()
  (fc1): Dense(input_channels=3072, output_channels=512, has_bias=True)
  (relu1): ReLU()
  (fc2): Dense(input_channels=512, output_channels=512, has_bias=True)
  (relu2): ReLU()
  (out): Dense(input_channels=512, output_channels=10, has_bias=True)
)


RuntimeError: Exception thrown from dataset pipeline. Refer to 'Dataset Pipeline Error Message'. 

------------------------------------------------------------------
- Dataset Pipeline Error Message: 
------------------------------------------------------------------
[ERROR] Invalid cifar folder, cifar(.bin) files are missing under ./dataset.

------------------------------------------------------------------
- C++ Call Stack: (For framework developers) 
------------------------------------------------------------------
mindspore/ccsrc/minddata/dataset/engine/datasetops/source/cifar_op.cc(223).


